In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Sparkify").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 17:33:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_file = "data/mini_sparkify_event_data.json"

In [4]:
df = spark.read.json(data_file)

In [5]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

In [6]:
table_cols = df.columns

In [7]:
table_cols
# ['artist',
#  'auth',
#  'firstName',
#  'gender',
#  'itemInSession',
#  'lastName',
#  'length',
#  'level',
#  'location',
#  'method',
#  'page',
#  'registration',
#  'sessionId',
#  'song',
#  'status',
#  'ts',
#  'userAgent',
#  'userId']

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [8]:
colTypes = [f.dataType for f in df.schema.fields]

In [9]:
colTypes
# [StringType(),
#  StringType(),
#  StringType(),
#  StringType(),
#  LongType(),
#  StringType(),
#  DoubleType(),
#  StringType(),
#  StringType(),
#  StringType(),
#  StringType(),
#  LongType(),
#  LongType(),
#  StringType(),
#  LongType(),
#  LongType(),
#  StringType(),
#  StringType()]

[StringType(),
 StringType(),
 StringType(),
 StringType(),
 LongType(),
 StringType(),
 DoubleType(),
 StringType(),
 StringType(),
 StringType(),
 StringType(),
 LongType(),
 LongType(),
 StringType(),
 LongType(),
 LongType(),
 StringType(),
 StringType()]

In [10]:
userIdCol = "userId"
timeCol = "ts"

In [11]:
from pyspark.sql.functions import col, collect_list, struct

In [12]:
s = struct([col(x) for x in table_cols])

In [13]:
collect = collect_list(s).alias("data")

In [14]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

In [15]:
grouped = df.groupBy(userIdCol).agg(collect)

In [16]:
from pyspark.sql.types import StructType, StructField, LongType, ArrayType

In [17]:
group51 = grouped.filter(col(userIdCol) == 51).collect()[0]

24/04/11 17:33:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
def sort_group_by_ts(structure):
    return sorted(structure, key=lambda x: x.ts)

In [19]:
from pyspark.sql.functions import udf

In [20]:
# the group's schema is the same as the original dataframe
group_schema = df.schema

In [21]:
sort_udf = udf(sort_group_by_ts, ArrayType(group_schema))

In [22]:
sorted_df = grouped.withColumn("data", sort_udf(col("data")))

In [23]:
userIds = sorted_df.select(userIdCol).collect()

In [24]:
for id in userIds:
    break

In [25]:
id.userId

'51'

In [26]:
def get_user_data(id):
    return sorted_df.filter(col(userIdCol) == id.userId).collect()[0]

In [27]:
from tqdm.auto import tqdm

In [28]:
fewest_id = None
fewest_events = float("inf")
for id in tqdm(userIds):
    user_data = get_user_data(id)
    num_events = len(user_data.data)
    if num_events < fewest_events:
        fewest_id = id
        fewest_events = num_events

  0%|          | 0/226 [00:00<?, ?it/s]

In [34]:
fewest = get_user_data(fewest_id)

In [35]:
fewest

Row(userId='135', data=[Row(artist='Coldplay', auth='Logged In', firstName='Zoey', gender='F', itemInSession=0, lastName='Nelson', length=311.27465, level='free', location='Las Vegas-Henderson-Paradise, NV', method='PUT', page='NextSong', registration=1532433959000, sessionId=134, song='The Scientist', status=200, ts=1538661151000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"', userId='135'), Row(artist='Ratatat', auth='Logged In', firstName='Zoey', gender='F', itemInSession=1, lastName='Nelson', length=226.53342, level='free', location='Las Vegas-Henderson-Paradise, NV', method='PUT', page='NextSong', registration=1532433959000, sessionId=134, song='Loud Pipes', status=200, ts=1538661462000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"', userId='135'), Row(artist='Fergie', auth='Logged In', firstName='Zoey', gender='F', itemInSession

In [40]:
fewest_filtered = sorted_df.filter(sorted_df.userId == '135')

In [44]:
fewest_filtered.write.format("json").save("data/fewest_events")